In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import datetime as dt

In [ ]:
#User inputs
#Start and end dates 
#Has to be in yyyy-mm-dd format (ex: 2015-01-01)
startDate = 'xxxx'
endDate = 'xxxx'

#name of the location (i.e. axial_base, oregon_offshore, oregon_shelf, oregon_slope)
locationName1 = "xxxx"
locationName2 = "xxxx"
locationName3 = "xxxx"
locationName4 = "xxxx"

focusLocation = "xxxx"

#directory where the pkl files are stored (for windows computers, may need to use \\ when specifying directory. Ex: C:\\Users\\)
directory = "xxxx"

#export directory path (Where you want an excel file placed to make data grabbing quicker)
exportDirectory = "xxxx"

In [ ]:
# function to obtain files
def getData(path):

    sound_speed = []
    depth_mean = []
    dates = []

    for file in os.listdir(path):
        with open(file, 'rb') as f:
            loadFile = pickle.load(f)

            # Get date for 1D array
            date = file.replace(".pkl","")
            #dates into an array
            dates.append(date)

            # Speed of sound put into array
            sound_speed.append(loadFile.parameter_mean)

            # Depth_mean into array
            depth_mean.append(loadFile.depth_mean) 
            
    f.close()

    dates = np.array(dates)
    sound_speed = np.array(sound_speed)
    depth_mean = np.array(depth_mean)
    return dates, depth_mean, sound_speed

In [ ]:
# interpolation function
# interpolates values between two numbers 
# input: 2 1D arrays: 1 is depth, 1 is sound speed
# output: 1D array of interpolated sound speeds with respect to bin sizes of the depth (1m in this case)
# make new empty array with dimensions of the depth (3000 in this case)
# make equal size bins (1m)
# if you can interpolate, then insert interpolated value, if not: then append.NaN

def soundSpeedInterpolation(depth, soundSpeed):

    # interpolation conditions
    # if not NaN & i+1 is not NaN -> interpolate btwn i and i+1
    # else -> NaN
    
    interpValues = []
    jstart = 0
    
    #i goes through all the depths of set bins of 0-2999 depths each by seperated by 1m
    for i in range (0,3000):
        
        #checks if the next depth has no data, if no data, then insert a nan value since we can skip the interpolation function
        if np.isnan(depth[jstart+1]):
            interpValues.append(np.nan)
        else:
            #found represents if an interpolated depth has been found
            found = False
            lendepth = len(depth)-1
            
            # target depth
            x = i
            
            # checking if nan value in the depth array, if it hits a nan value, then immediately goes to i array to fill 
            # rest of array with nan values
             
            # j goes through the array of actual depths and interpolating based on those depths
            # tries to interpolate each specified depth, but if it can't find the specified in the data, then insert
            # nan value, then check for the next depth
            
            for j in range (jstart, lendepth):
                
                
                #if the current value nan value, then break and insert nan value
                if np.isnan(depth[j]):
                    break

                # for efficiency of the process
                lowerDepth = depth[j]
                upperDepth = depth[j+1]
                lowerSpeed = soundSpeed[j]
                upperSpeed = soundSpeed[j+1]
                
                # interpolation function 
                if x >= lowerDepth and x < upperDepth and upperDepth - lowerDepth < 2:
                    slope = (upperSpeed - lowerSpeed) / (upperDepth - lowerDepth)

                    # interpolation
                    interpSoundSpeed = slope * (x - lowerDepth) + lowerSpeed
                    
                    found = True
                    
                    # makes loops more efficient by last known value
                    jstart = j+1
                    break
            
            if found:
                interpValues.append(interpSoundSpeed)
            else: 
                interpValues.append(np.nan)
            
    interpValues = np.array(interpValues)
    return interpValues

In [ ]:
# Organize data into 2D array (Columns = Dates, Rows = Depth, each entry = sound speed)
# Interpolate sound speed at desired depths
# plot 2D array using contourf
# contourf args: X = 1D array of dates, Y = 1D array of depths, Z = 2D array of sound speeds @ Depth & Date
# N = 2193, M = 3000

def organizeData(depth_mean, sound_speed):

    Z = []
    
    # columns = dates (i), rows = depth (j)
    # Get Z values
    for i in range(0,len(depth_mean)):
        Z.append(soundSpeedInterpolation(depth_mean[i], sound_speed[i]))

    Z = np.array(Z)
    return Z

In [ ]:

#export to excel file, can readback from excel to avoid interpolation function -> speeding up the code
def readWriteData(locationName):
    excelFile = locationName + ".xlsx"
    excelPath = exportDirectory
    excelDirectory = excelPath + excelFile
    
    path = directory + "\\" + locationName
    os.chdir(path)

    #if the file already exists
    if os.path.exists(excelDirectory):
        data = pd.read_excel(excelDirectory)
    else:
        #get data
        dataTuple = getData(path)
        dates = dataTuple[0]
        depth_mean = dataTuple[1]
        sound_speed = dataTuple[2]

        #organize data into pandas dataframe
        rawData = organizeData(depth_mean, sound_speed)
        data = rawData.T
        data = pd.DataFrame(data)

        #sets dates for the data in the pandas dataframe
        data = data.set_axis(dates, axis=1, inplace=False)

        #export to excel
        os.chdir(excelPath)
        data.to_excel(excelFile, sheet_name="sheet1", index = False, na_rep='NaN')
    return data

In [ ]:
def getDateData(data, startDate, endDate):
    #get dates
    dates = pd.date_range(start=startDate, end=endDate)
    
    #get indexs of the dates in data
    dateColumns = data.columns
    startIndex = dateColumns.get_loc(startDate)
    endIndex = dateColumns.get_loc(endDate)+1
    
    #shorten data to include values inbetween and including start and end date
    dateData = data.iloc[:,startIndex:endIndex]
    numColumns = len(dateData.columns)
    
    return dateData, dates, numColumns

In [ ]:
# find local maxima function
# finds local maxima in the first 200m range area
# returns the depth of the max sound speed and the max sound speed
def localMaxima(dayData):
    
    localMaximaDepth = 0
    maxSoundSpeed = 0
    
    for i in range(0,201):
        if not np.isnan(dayData[i]) and dayData[i] > maxSoundSpeed:
            localMaximaDepth = i
            maxSoundSpeed = dayData[i]
            
    if localMaximaDepth == 0:
        localMaximaDepth = np.nan
            
    return localMaximaDepth, maxSoundSpeed

In [ ]:
#converts dates into strings to be used in pandas data fetching
def convertDate(date):
    year = str(date.date().year)
        
    month = date.date().month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
        
    day = date.date().day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    #combines values into a date usable by Pandas 
    date = year + '-' + month + '-' + day
    return date

In [ ]:
#10m to local maxima data storage
def ThirtyMAndLocalMaxima(data, startDate, endDate):
    #get dates & date data
    datesTuple = getDateData(data, startDate, endDate)
    
    #set dateTuple data to variables
    dateData = datesTuple[0]
    dates = datesTuple[1]
    numColumns = datesTuple[2]
    
    #set up arrays
    TenM = []
    speedLocalMaxima = []
    depthLocalMaxima = []
    
    #loop through the dates
    #add local maximas and speed of sound at 10m to arrays
    for i in range(0, numColumns):
        
        date = convertDate(dates[i])
        
        #adds speed of sound at 10m and local maximas to lists
        TenMSpeed = dateData[date].values[30]
        TenM.append(TenMSpeed)
        
        localMax = 0
        
        localMax = localMaxima(dateData[date])
        
        LocalMaximaDepth = localMax[0]
        depthLocalMaxima.append(LocalMaximaDepth)
        
        LocalMaximaSpeed = localMax[1]
        speedLocalMaxima.append(LocalMaximaSpeed)
    
    #sets up dictionary for the lists, then converts it to a pandas dataframe and returns it
    d = {'Dates': dates, 'Depth Local Maxima': depthLocalMaxima, 'Speed Local Maxima': speedLocalMaxima, '30m Sound Speed': TenM,}
    storeData = pd.DataFrame(data=d)
    return storeData

In [ ]:
#only works for 30 meter data 
def excludeFlat(data):
    tracker = []
    
    depthData = data["Depth Local Maxima"].to_numpy()
    
    for i in range(0,len(depthData)):
        
        pt = depthData[i]
        
        if i == len(depthData)-1:
            tracker.append(True)
            break
        
        if abs(depthData[i]-depthData[i+1]) <= 3:
            tracker.append(False)
        else:
            tracker.append(True)
            
    newData = data.loc[tracker,:]
    
    return newData

In [ ]:
def averageSoundSpeed(data, startDate, endDate):
    #get dates & date data
    datesTuple = getDateData(data, startDate, endDate)
    
    #set dateTuple data to variables
    dateData = datesTuple[0]
    dates = datesTuple[1]
    numColumns = datesTuple[2]
    
    #set up arrays
    speedAvgs = []
    
    #goes through every depth and gets average speed
    for i in range (0,201):
        
        speedSum = 0
        countNaN = 0
        
        for j in range(0, numColumns):
            date = convertDate(dates[j])
            
            speed = dateData[date].values[i]
            
            if not np.isnan(speed):
                speedSum = speedSum + speed
            else:
                countNaN = countNaN + 1
        if speedSum == 0:
            speedAvgs.append(np.nan)
        else:
            speedAvg = speedSum / (numColumns - countNaN)
            speedAvgs.append(speedAvg)
            
    #sets up dictionary for the lists, then converts it to a pandas dataframe and returns it
    depth = list(range(0,201))
    d = {'Average Speed of Sound': speedAvgs, 'Depth': depth}
    storeData = pd.DataFrame(data=d)
    return storeData

In [ ]:
#extrapolation function to get the sound speed
#takes in dateData
#extrapolates data at 30m 
#uses linear extrapolation function because data is mostly linear in the 0 to 50m range
#returns a tuple of data: depth, sound speed
def extrapolateSoundSpeed(data, date):
    #check if data exist at 30m
    depth = 30
    soundSpeed = data[date].values[depth]
    if not np.isnan(soundSpeed):
        return depth, soundSpeed
    
    #sets up for later data checks
    depthAbove30 = np.nan
    dataAbove30 = True
    depthBelow30 = np.nan
    dataBelow30 = True
    
    #10m tolerance as data likely won't be accurate outside of that range
    
    #check for data above 30m 
    for i in range (20,30):
        soundSpeed = data[date].values[i]
        if not np.isnan(soundSpeed):
            depthAbove30 = i
    
    if not np.isnan(depthAbove30):
        soundSpeedAbove30 = data[date].values[depthAbove30]
    else:
        dataAbove30 = False
    
    #check for data below 30m
    for j in range (31,40):
        soundSpeed = data[date].values[j]
        if not np.isnan(soundSpeed):
            depthBelow30 = j
            #need break to get closest data point to 30
            break
    
    if not np.isnan(depthBelow30):
        soundSpeedBelow30 = data[date].values[depthBelow30]
    else:
        dataBelow30 = False
        
    if dataAbove30 == False and dataBelow30 == False:
        depth = np.nan
        soundSpeed = np.nan
        return depth, soundSpeed
    
    #interpolation if data above and below 30m
    if not np.isnan(depthAbove30) and not np.isnan(depthBelow30):
        slope = (soundSpeedAbove30 - soundSpeedBelow30) / (depthAbove30 - depthBelow30)
        interpSoundSpeed = slope * (35 - depthBelow30) + soundSpeedBelow30
        depth = 35
        return depth, interpSoundSpeed
    
    #extrapolation above 30m
    if not np.isnan(depthAbove30) and np.isnan(depthBelow30):
        
        slope = np.nan
        exterpSoundSpeed = np.nan
        
        for i in range(20,30):
            current = data[date].values[i]
            infront = data[date].values[i + 1]
            
            if not np.isnan(infront):
                slope = (infront - current) / ((i+1) - i)
                exterpSoundSpeed = slope * (30 - i) + current
        
        return slope, exterpSoundSpeed
    
    #extrapolation below 30m
    if np.isnan(depthAbove30) and not np.isnan(depthBelow30):
        
        slope = np.nan
        exterpSoundSpeed = np.nan
        
        for i in range(31,40):
            current = data[date].values[i]
            infront = data[date].values[i + 1]
            
            if not np.isnan(infront):
                slope = (infront - current) / ((i+1) - i)
                exterpSoundSpeed = slope * (30 - i) + current
                #return here because, need slope closest to 30m
                return slope, exterpSoundSpeed

    #no fullfills no conditions? Return nan
    depth = np.nan
    soundSpeed = np.nan
    return depth, soundSpeed

In [ ]:
#Sound channel dimensions finder
#takes in the base data and uses the ThirtyMAndLocalMaxima function to help find the sound channel dimensions
def soundChannelDim(data, startDate, endDate):
    #get dates & date data
    datesTuple = getDateData(data, startDate, endDate)
    
    #set dateTuple data to variables
    dateData = datesTuple[0]
    dates = datesTuple[1]
    numColumns = datesTuple[2]
    
    ThirtyM_LMM_Data = ThirtyMAndLocalMaxima(data, startDate, endDate)
    
    lengthArray = []
    widthArray = []
    
    for i in range(0,ThirtyM_LMM_Data.shape[0]):
        #length calculation
        length = np.nan
        width = np.nan
        
        date = convertDate(dates[i])
        
        shallowTuple = extrapolateSoundSpeed(dateData, date)
        
        shallowDepth = shallowTuple[0]
        shallowSoundSpeed = shallowTuple[1]
        
        if not np.isnan(shallowDepth):
            localMaximaDepth = ThirtyM_LMM_Data['Depth Local Maxima'].values[i]
            length = localMaximaDepth - shallowDepth
            
        lengthArray.append(length)
        
        #width calculation
        if not np.isnan(shallowSoundSpeed):
            localMaximaSoundSpeed = ThirtyM_LMM_Data['Speed Local Maxima'].values[i]
            width = localMaximaSoundSpeed - shallowSoundSpeed
            
        widthArray.append(width)

    d = {'Dates': dates, 'Sound Channel Length': lengthArray, 'Sound Channel Width': widthArray}
    storeData = pd.DataFrame(data=d)
    return storeData        

In [ ]:
#Load Data for all 4 datasets
location1 = readWriteData(locationName1)
location2 = readWriteData(locationName2)
location3 = readWriteData(locationName3)
location4 = readWriteData(locationName4)


In [ ]:
#focus data set
focusData = location1

if (focusLocation == locationName2):
    focusData = location2
elif (focusLocation == locationName3):
    focusData = location3
else:
    focusData = location4

In [ ]:
#average speed of sound
avgSoundSpeed = averageSoundSpeed(focusData, startDate, endDate)
soundSpeedScatter = hv.Scatter(avgSoundSpeed)
soundSpeedScatter.opts(invert_yaxis=True, 
             width=800, height=500,
             tools = ['hover'],
             title = "Average Sound Speed from " + startDate + " to " + endDate + " for "+ focusLocation)
soundSpeedScatter

In [ ]:
#calculate 30m and local max and min
location1_30mLMM_Data = ThirtyMAndLocalMaxima(location1, startDate, endDate)
location2_30mLMM_Data = ThirtyMAndLocalMaxima(location2, startDate, endDate)
location3_30mLMM_Data = ThirtyMAndLocalMaxima(location3, startDate, endDate)
location4_30mLMM_Data = ThirtyMAndLocalMaxima(location4, startDate, endDate)


In [ ]:
#remove flat data portions
location1_truncated = excludeFlat(location1_30mLMM_Data)
location2_truncated = excludeFlat(location2_30mLMM_Data)
location3_truncated = excludeFlat(location3_30mLMM_Data)
location4_truncated = excludeFlat(location4_30mLMM_Data)

In [ ]:
#create scatter plots
#Multiple plots reference: https://justinbois.github.io/bootcamp/2020/lessons/l27_holoviews.html
#Another reference: http://holoviews.org/user_guide/Composing_Elements.html

location1_Scatter = hv.Scatter(location1_truncated, label = locationName1)
location2_Scatter = hv.Scatter(location2_truncated, label = locationName2)
location3_Scatter = hv.Scatter(location3_truncated, label = locationName3)
location4_Scatter = hv.Scatter(location4_truncated, label = locationName4)

In [ ]:
ThirtyM_And_LM_Plots = location1_Scatter * location2_Scatter * location3_Scatter * location4_Scatter
ThirtyM_And_LM_Plots.opts(
    invert_yaxis=True, 
    width=800, height=500,
    tools = ['hover'],
    title = "Sound Speed Local Maxima at Depth"
)

In [ ]:
#sound channel dimensions
#location1
location1_soundChannelDim = soundChannelDim(location1, startDate, endDate)
location1_soundChannelLength = location1_soundChannelDim[["Dates", "Sound Channel Length"]]
location1_soundChannelWidth = location1_soundChannelDim[["Dates", "Sound Channel Width"]]

#location2
location2_soundChannelDim = soundChannelDim(location2, startDate, endDate)
location2_soundChannelLength = location2_soundChannelDim[["Dates", "Sound Channel Length"]]
location2_soundChannelWidth = location2_soundChannelDim[["Dates", "Sound Channel Width"]]

#location3
location3_soundChannelDim = soundChannelDim(location3, startDate, endDate)
location3_soundChannelLength = location3_soundChannelDim[["Dates", "Sound Channel Length"]]
location3_soundChannelWidth = location3_soundChannelDim[["Dates", "Sound Channel Width"]]

#location4
location4_soundChannelDim = soundChannelDim(location4, startDate, startDate)
location4_soundChannelLength = location4_soundChannelDim[["Dates", "Sound Channel Length"]]
location4_soundChannelWidth = location4_soundChannelDim[["Dates", "Sound Channel Width"]]

In [ ]:
location1_Scatter_L = hv.Scatter(location1_soundChannelLength, label = locationName1)
location2_Scatter_L = hv.Scatter(location2_soundChannelLength, label = locationName2)
location3_Scatter_L = hv.Scatter(location3_soundChannelLength, label = locationName3)
location4_Scatter_L = hv.Scatter(location4_soundChannelLength, label = locationName4)

soundChannelLengthPlots = location1_Scatter_L * location2_Scatter_L * location3_Scatter_L * location4_Scatter_L
soundChannelLengthPlots.opts(
                        invert_yaxis=True,
                        width=800, height=500,
                        tools = ['hover'],
                        title = "Sound Channel Length")

In [ ]:
location1_Scatter_W = hv.Scatter(location1_soundChannelWidth, label = locationName1)
location2_Scatter_W = hv.Scatter(location2_soundChannelWidth, label = locationName2)
location3_Scatter_W = hv.Scatter(location3_soundChannelWidth, label = locationName3)
location4_Scatter_W = hv.Scatter(location4_soundChannelWidth, label = locationName4)

soundChannelWidthPlots = location1_Scatter_W * location2_Scatter_W * location3_Scatter_W * location4_Scatter_W
soundChannelWidthPlots.opts(
                        invert_yaxis=True,
                        width=800, height=500,
                        tools = ['hover'],
                        title = "Sound Channel Width")